In [1]:
import pandas as pd
import dlib
import cv2
import numpy as np
import os

In [2]:
def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((shape.num_parts, 2), dtype=dtype)

    # loop over all facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, shape.num_parts):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords

In [3]:
def get_rect(shape):
    rw = 0
    rh = 0
    rx = 65535
    ry = 65535
    for (x,y) in shape:
        rw = max(rw,x) # w
        rh = max(rh,y) # h
        rx = min(rx,x) # x
        ry = min(ry,y) # y
    return (rx,ry,rw-rx,rh-ry)

In [6]:
######################################
# person & sentences
person = '00' 
sentence= '00' 
num = '00' # 반복 횟수
#######################################
count = 0
dataset = {}
dataset[person] = {}
dataset[person][sentence] ={}
dataset[person][sentence][num] ={}

# create face detector, predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("data/shape_predictor_68_face_landmarks.dat") # predict 해주는애            

cap = cv2.VideoCapture(0)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out_path = 'test/%s/video' %(person)
try:
    os.makedirs(out_path)
except:
    pass
out_name = f'video_{sentence}_{num}.avi'
out_path = out_path + '/' + out_name
out = cv2.VideoWriter(out_path, fourcc, 5.0, (640, 480))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        image = cv2.resize(frame, dsize = (640, 480), interpolation = cv2.INTER_AREA)
        
        img = image.copy()
        img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        out.write(image)
        
        face_detector = detector(img_gray, 1)
        if len(face_detector) == 0:
            print("***************No face Detected*************,%d", int(cap.get(1)))
        for face in face_detector:
            # cropping face
            crop_face = image.copy()
            crop_face = crop_face[face.top():face.bottom(), face.left():face.right()]
            # make prediction and transform to numpy array
            landmarks = predictor(image, face)
            landmarks = shape_to_np(landmarks)
            lip_landmarks=[landmarks[x] for x in range(48, 68)]
            image_crop = image.copy()
            (x, y, w, h) = get_rect(lip_landmarks)
            crop_image = image_crop[y:y+h, x:x+w]
            
            for(i,(x_i, y_i)) in enumerate(lip_landmarks):
                cv2.circle(image, (x_i, y_i), 1, (0,0,255), -1)
            
            (x, y, w, h) = get_rect(lip_landmarks)
            for s in lip_landmarks:
                s[0] -= x
                s[1] -= y
                
            
        cv2.imshow('frame',image)

        dataset[person][sentence][num][count] = {}
        if(int(cap.get(1))%3  == 0): 
            path = 'test/%s/%s/%s/image' %(person, sentence, num)
            crop_path = 'test/%s/%s/%s/crop_lip' %(person, sentence, num)
            crop_path2 = 'test/%s/%s/%s/crop_face' %(person, sentence, num)
            
            # path 없다면 만들고 있다면 pass
            try:
                os.makedirs(path)
                os.makedirs(crop_path)
                os.makedirs(crop_path2)
            except FileExistsError as e:
                pass
            img_name = f'img_{count}.jpg'
            mouth_name = f'mouth_{count}.jpg'
            face_name = f'face_{count}.jpg'
            img_path = path + '/' + img_name
            mouth_path = crop_path + '/' + mouth_name
            face_path = crop_path2 + '/' + face_name
            
            # save images as frame
            print(img_path)
            print(img)
            
            cv2.imwrite(img_path, img)
            cv2.imwrite(mouth_path, crop_image)
            cv2.imwrite(face_path, crop_face)
            print(lip_landmarks)
            print('Saved Frame number: ' + str(int(cap.get(1)))+ '\n')
            dataset[person][sentence][num][count] = lip_landmarks
            count +=  1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
cap.release()
out.release()
cv2.destroyAllWindows()

test/00/00/00/image/img_0.jpg
[[[ 68  63  82]
  [ 66  64  80]
  [ 61  62  76]
  ...
  [ 87  99 125]
  [ 88  97 126]
  [ 87  96 125]]

 [[ 75  70  91]
  [ 72  69  90]
  [ 66  66  86]
  ...
  [ 85 100 122]
  [ 85 100 123]
  [ 83  98 122]]

 [[ 67  72  92]
  [ 65  69  89]
  [ 60  65  85]
  ...
  [ 80 100 124]
  [ 84 101 125]
  [ 85 102 126]]

 ...

 [[ 46  71  90]
  [ 47  73  90]
  [ 47  73  91]
  ...
  [ 61  86 111]
  [ 63  85 108]
  [ 58  79 102]]

 [[ 43  73  91]
  [ 46  74  91]
  [ 48  74  92]
  ...
  [ 54  80 108]
  [ 53  76 104]
  [ 51  74 101]]

 [[ 43  72  94]
  [ 47  73  95]
  [ 49  71  94]
  ...
  [ 52  75 102]
  [ 46  69  97]
  [ 46  69  97]]]
[array([ 0, 18]), array([8, 7]), array([17,  1]), array([23,  2]), array([29,  0]), array([38,  6]), array([47, 17]), array([39, 22]), array([30, 25]), array([23, 25]), array([17, 25]), array([ 9, 23]), array([ 4, 17]), array([17, 12]), array([23, 12]), array([29, 12]), array([43, 16]), array([29, 13]), array([23, 14]), array([17, 13])]
S

test/00/00/00/image/img_10.jpg
[[[ 53  60  80]
  [ 50  56  76]
  [ 54  59  79]
  ...
  [ 79 105 117]
  [ 79 104 116]
  [ 77 102 113]]

 [[ 69  70  88]
  [ 60  63  81]
  [ 57  61  81]
  ...
  [ 86 105 117]
  [ 87 105 117]
  [ 87 105 117]]

 [[ 77  73  91]
  [ 69  70  88]
  [ 63  68  85]
  ...
  [ 87 105 115]
  [ 87 105 117]
  [ 87 105 117]]

 ...

 [[ 47  74  85]
  [ 46  75  86]
  [ 44  76  86]
  ...
  [ 67  85 115]
  [ 63  84 114]
  [ 63  84 114]]

 [[ 46  72  88]
  [ 48  75  88]
  [ 48  75  86]
  ...
  [ 59  80 110]
  [ 56  77 107]
  [ 52  72 102]]

 [[ 48  74  90]
  [ 48  74  90]
  [ 49  75  91]
  ...
  [ 55  78 108]
  [ 49  72 102]
  [ 47  70 100]]]
[array([ 0, 26]), array([6, 9]), array([16,  0]), array([22,  1]), array([29,  0]), array([39,  9]), array([45, 26]), array([39, 40]), array([30, 46]), array([22, 46]), array([15, 45]), array([ 6, 39]), array([ 4, 25]), array([16, 11]), array([22, 10]), array([28, 11]), array([41, 25]), array([29, 34]), array([22, 35]), array([16, 34])]


test/00/00/00/image/img_20.jpg
[[[ 50  66  83]
  [ 50  61  78]
  [ 57  62  80]
  ...
  [100 101 125]
  [ 98  97 122]
  [ 97  96 121]]

 [[ 53  70  87]
  [ 53  65  83]
  [ 55  61  81]
  ...
  [ 98 100 123]
  [ 97  97 117]
  [ 99  98 118]]

 [[ 53  69  88]
  [ 55  69  90]
  [ 55  66  90]
  ...
  [ 92 100 119]
  [ 89 100 120]
  [ 88 100 120]]

 ...

 [[ 48  72  88]
  [ 48  71  89]
  [ 48  71  90]
  ...
  [ 58  86 114]
  [ 57  85 112]
  [ 54  82 109]]

 [[ 48  72  88]
  [ 48  71  88]
  [ 48  71  90]
  ...
  [ 57  82 111]
  [ 56  79 109]
  [ 54  77 107]]

 [[ 48  71  90]
  [ 48  71  91]
  [ 49  71  94]
  ...
  [ 50  78 106]
  [ 45  73 100]
  [ 44  72  99]]]
[array([ 0, 19]), array([8, 7]), array([17,  0]), array([23,  2]), array([29,  0]), array([38,  7]), array([45, 20]), array([38, 29]), array([29, 33]), array([23, 34]), array([16, 33]), array([ 7, 29]), array([ 4, 18]), array([17, 11]), array([23, 11]), array([29, 11]), array([41, 19]), array([29, 22]), array([23, 22]), array([17, 21])]


test/00/00/00/image/img_30.jpg
[[[ 80  73  94]
  [ 77  71  92]
  [ 72  67  88]
  ...
  [ 67 105 123]
  [ 67 103 125]
  [ 67 103 125]]

 [[ 76  71  92]
  [ 73  69  90]
  [ 69  64  85]
  ...
  [ 67 104 120]
  [ 67 104 121]
  [ 67 104 121]]

 [[ 75  70  91]
  [ 70  68  88]
  [ 63  63  84]
  ...
  [ 80 103 119]
  [ 77 103 119]
  [ 77 103 119]]

 ...

 [[ 39  74  89]
  [ 39  74  89]
  [ 37  73  88]
  ...
  [ 52  82 110]
  [ 47  82 108]
  [ 50  85 112]]

 [[ 40  75  92]
  [ 39  73  91]
  [ 36  71  88]
  ...
  [ 44  76 106]
  [ 47  79 108]
  [ 53  85 114]]

 [[ 39  73  91]
  [ 38  73  91]
  [ 37  72  90]
  ...
  [ 40  72 101]
  [ 43  75 104]
  [ 44  77 105]]]
[array([ 0, 17]), array([8, 7]), array([16,  1]), array([21,  2]), array([26,  0]), array([34,  6]), array([40, 17]), array([34, 23]), array([26, 26]), array([21, 27]), array([15, 26]), array([ 7, 23]), array([ 5, 16]), array([15, 10]), array([21, 11]), array([26, 10]), array([36, 16]), array([26, 15]), array([21, 16]), array([16, 15])]


test/00/00/00/image/img_40.jpg
[[[ 66  73  82]
  [ 65  70  78]
  [ 61  64  72]
  ...
  [ 68 105 112]
  [ 71 102 113]
  [ 68  99 110]]

 [[ 65  72  81]
  [ 64  69  76]
  [ 59  61  70]
  ...
  [ 73 105 115]
  [ 73 106 112]
  [ 71 104 109]]

 [[ 67  68  82]
  [ 64  67  80]
  [ 60  66  79]
  ...
  [ 72 105 114]
  [ 73 106 112]
  [ 74 107 113]]

 ...

 [[ 44  74  92]
  [ 44  74  92]
  [ 44  74  92]
  ...
  [ 45  89 114]
  [ 41  88 113]
  [ 42  89 114]]

 [[ 37  74  91]
  [ 38  74  94]
  [ 39  72  94]
  ...
  [ 40  86 113]
  [ 38  84 111]
  [ 36  82 110]]

 [[ 39  76  93]
  [ 37  75  92]
  [ 35  72  90]
  ...
  [ 38  85 111]
  [ 36  83 108]
  [ 35  82 107]]]
[array([ 0, 14]), array([10,  5]), array([19,  0]), array([25,  2]), array([31,  1]), array([41,  6]), array([49, 15]), array([40, 21]), array([31, 23]), array([24, 24]), array([18, 23]), array([ 9, 20]), array([ 4, 14]), array([19, 10]), array([25, 11]), array([31, 11]), array([45, 14]), array([31, 12]), array([25, 13]), array([19, 12])

In [8]:
# csv 파일은 문장 폴더 안에 입모양 반복 횟수대로 이름지어서 저장됨
fileCount = 0
shapes = []
for count in dataset[person][sentence][num].keys():
    shape = dataset[person][sentence][num][count]
    for (x,y) in shape:
        shapes.append((num,count,x,y))

df = pd.DataFrame(data = shapes,columns = ['num','frame','x','y'])
path = "csv"
try:
    os.makedirs(path)
except FileExistsError as e:
    pass
filename = "%s_%s_%s.csv" %(person, sentence, num)
csvPath = "%s/%s" % (path,filename)
print("saving %s" % csvPath)
df.to_csv(csvPath)

saving csv/00_00_02.csv
